In [2]:
import requests
import json
import time
from datetime import datetime
import pandas as pd
import os

class GroundhoggAPI:
    def __init__(self, site_url, token, public_key):
        """
        Initialize the Groundhogg API client
        
        Args:
            site_url (str): Your WordPress site URL (e.g., https://example.com)
            token (str): Your Groundhogg API token
            public_key (str): Your Groundhogg public key
        """
        self.base_url = f"{site_url.rstrip('/')}/wp-json/gh/v4"
        self.token = token
        self.public_key = public_key
        self.headers = {
            "Gh-Token": self.token,
            "Gh-Public-Key": self.public_key
        }
        
    def _make_request(self, endpoint, method="GET", params=None, data=None):
        """Make an API request to Groundhogg"""
        url = f"{self.base_url}/{endpoint.lstrip('/')}"
        
        try:
            if method.upper() == "GET":
                response = requests.get(url, headers=self.headers, params=params)
            elif method.upper() == "POST":
                response = requests.post(url, headers=self.headers, json=data, params=params)
            elif method.upper() == "PUT":
                response = requests.put(url, headers=self.headers, json=data, params=params)
            elif method.upper() == "DELETE":
                response = requests.delete(url, headers=self.headers, params=params)
            else:
                raise ValueError(f"Unsupported HTTP method: {method}")
            
            response.raise_for_status()  # Raise exception for HTTP errors
            return response.json()
        
        except requests.exceptions.RequestException as e:
            print(f"API request error: {e}")
            return None
    
    def get_contacts(self, page=1, per_page=100):
        """
        Get contacts from Groundhogg
        
        Args:
            page (int): Page number for pagination
            per_page (int): Number of contacts per page
            
        Returns:
            dict: JSON response containing contacts
        """
        params = {
            'page': page,
            'per_page': per_page
        }
        return self._make_request('/contacts', params=params)
    
    def get_all_contacts(self):
        """
        Get all contacts from Groundhogg by paginating through results
        
        Returns:
            list: List of all contacts
        """
        all_contacts = []
        page = 1
        per_page = 100
        
        while True:
            response = self.get_contacts(page=page, per_page=per_page)
            
            if not response or 'contacts' not in response or not response['contacts']:
                break
                
            all_contacts.extend(response['contacts'])
            
            if len(response['contacts']) < per_page:
                break
                
            page += 1
            time.sleep(1)  # Avoid hitting rate limits
        
        return all_contacts
    
    def get_tags(self, page=1, per_page=100):
        """Get all tags"""
        params = {
            'page': page,
            'per_page': per_page
        }
        return self._make_request('/tags', params=params)
    
    def get_all_tags(self):
        """Get all tags by paginating through results"""
        all_tags = []
        page = 1
        per_page = 100
        
        while True:
            response = self.get_tags(page=page, per_page=per_page)
            
            if not response or 'tags' not in response or not response['tags']:
                break
                
            all_tags.extend(response['tags'])
            
            if len(response['tags']) < per_page:
                break
                
            page += 1
            time.sleep(1)
        
        return all_tags
    
    def get_contact_meta(self, contact_id):
        """Get metadata for a specific contact"""
        return self._make_request(f'/contacts/{contact_id}/meta')
    
    def get_funnels(self):
        """Get all funnels"""
        return self._make_request('/funnels')
    
    def get_funnel_steps(self, funnel_id):
        """Get steps for a specific funnel"""
        return self._make_request(f'/funnels/{funnel_id}/steps')
    
    def get_events(self, page=1, per_page=100):
        """Get contact events"""
        params = {
            'page': page,
            'per_page': per_page
        }
        return self._make_request('/events', params=params)
    
    def get_all_events(self):
        """Get all events by paginating through results"""
        all_events = []
        page = 1
        per_page = 100
        
        while True:
            response = self.get_events(page=page, per_page=per_page)
            
            if not response or 'events' not in response or not response['events']:
                break
                
            all_events.extend(response['events'])
            
            if len(response['events']) < per_page:
                break
                
            page += 1
            time.sleep(1)
        
        return all_events
    
    def export_data_to_csv(self, data_type, file_name=None):
        """
        Export data to CSV file
        
        Args:
            data_type (str): Type of data to export (contacts, tags, events)
            file_name (str, optional): Custom file name
            
        Returns:
            str: Path to the saved CSV file
        """
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        if not file_name:
            file_name = f"groundhogg_{data_type}_{timestamp}.csv"
        
        data = []
        
        if data_type == 'contacts':
            data = self.get_all_contacts()
        elif data_type == 'tags':
            data = self.get_all_tags()
        elif data_type == 'events':
            data = self.get_all_events()
        else:
            raise ValueError(f"Unsupported data type: {data_type}")
        
        if not data:
            print(f"No {data_type} data found to export")
            return None
        
        df = pd.DataFrame(data)
        df.to_csv(file_name, index=False)
        
        print(f"Exported {len(data)} {data_type} to {file_name}")
        return file_name


def main():
    """Main function to run the script"""
    # Replace these with your actual credentials
    site_url = "https://scholistico.com"
    token = "26a0293b0a4da82a960e6eb9effb4e31"
    public_key = "0bc1ccfccb63183f99cbf2620847b64f"
    
    # Check if credentials are provided
    if token == "your_groundhogg_token" or public_key == "your_groundhogg_public_key":
        print("Please update the script with your actual API credentials.")
        return
    
    # Initialize the API client
    api = GroundhoggAPI(site_url, token, public_key)
    
    # Create directory for exports
    export_dir = "groundhogg_exports"
    os.makedirs(export_dir, exist_ok=True)
    os.chdir(export_dir)
    
    # Test the API connection
    print("Testing API connection...")
    test_response = api.get_contacts(page=1, per_page=1)
    if not test_response:
        print("Could not connect to the Groundhogg API. Please check your credentials and try again.")
        return
    
    print("API connection successful!")
    
    # Export all data types
    print("Starting data export from Groundhogg...")
    
    # Export contacts
    api.export_data_to_csv("contacts")
    
    # Export tags
    api.export_data_to_csv("tags")
    
    # Export events
    api.export_data_to_csv("events")
    
    print(f"All data exported to directory: {export_dir}")


if __name__ == "__main__":
    main()

Testing API connection...
API connection successful!
Starting data export from Groundhogg...
No contacts data found to export
No tags data found to export
No events data found to export
All data exported to directory: groundhogg_exports


In [3]:
import requests
import json
import time
from datetime import datetime
import pandas as pd
import os

class GroundhoggAPI:
    def __init__(self, site_url, token, public_key):
        """
        Initialize the Groundhogg API client
        
        Args:
            site_url (str): Your WordPress site URL (e.g., https://example.com)
            token (str): Your Groundhogg API token
            public_key (str): Your Groundhogg public key
        """
        self.base_url = f"{site_url.rstrip('/')}/wp-json/gh/v3"
        self.token = token
        self.public_key = public_key
        self.headers = {
            "Gh-Token": self.token,
            "Gh-Public-Key": self.public_key
        }
        
    def _make_request(self, endpoint, method="GET", params=None, data=None):
        """Make an API request to Groundhogg"""
        url = f"{self.base_url}/{endpoint.lstrip('/')}"
        
        try:
            print(f"Making request to: {url}")
            if method.upper() == "GET":
                response = requests.get(url, headers=self.headers, params=params)
            elif method.upper() == "POST":
                response = requests.post(url, headers=self.headers, json=data, params=params)
            elif method.upper() == "PUT":
                response = requests.put(url, headers=self.headers, json=data, params=params)
            elif method.upper() == "DELETE":
                response = requests.delete(url, headers=self.headers, params=params)
            else:
                raise ValueError(f"Unsupported HTTP method: {method}")
            
            response.raise_for_status()  # Raise exception for HTTP errors
            return response.json()
        
        except requests.exceptions.RequestException as e:
            print(f"API request error: {e}")
            if hasattr(e, 'response') and e.response is not None:
                print(f"Response content: {e.response.text}")
            return None
    
    def get_contacts(self, page=1, per_page=100):
        """
        Get contacts from Groundhogg
        
        Args:
            page (int): Page number for pagination
            per_page (int): Number of contacts per page
            
        Returns:
            dict: JSON response containing contacts
        """
        params = {
            'page': page,
            'per_page': per_page
        }
        return self._make_request('/contacts', params=params)
    
    def get_all_contacts(self):
        """
        Get all contacts from Groundhogg by paginating through results
        
        Returns:
            list: List of all contacts
        """
        all_contacts = []
        page = 1
        per_page = 100
        
        while True:
            response = self.get_contacts(page=page, per_page=per_page)
            
            if not response or 'contacts' not in response or not response['contacts']:
                break
                
            all_contacts.extend(response['contacts'])
            
            if len(response['contacts']) < per_page:
                break
                
            page += 1
            time.sleep(1)  # Avoid hitting rate limits
        
        return all_contacts
    
    def get_tags(self, page=1, per_page=100):
        """Get all tags"""
        params = {
            'page': page,
            'per_page': per_page
        }
        return self._make_request('/tags', params=params)
    
    def get_all_tags(self):
        """Get all tags by paginating through results"""
        all_tags = []
        page = 1
        per_page = 100
        
        while True:
            response = self.get_tags(page=page, per_page=per_page)
            
            if not response or 'tags' not in response or not response['tags']:
                break
                
            all_tags.extend(response['tags'])
            
            if len(response['tags']) < per_page:
                break
                
            page += 1
            time.sleep(1)
        
        return all_tags
    
    def get_companies(self, page=1, per_page=100):
        """Get all companies"""
        params = {
            'page': page,
            'per_page': per_page
        }
        return self._make_request('/companies', params=params)
    
    def get_all_companies(self):
        """Get all companies by paginating through results"""
        all_companies = []
        page = 1
        per_page = 100
        
        while True:
            response = self.get_companies(page=page, per_page=per_page)
            
            if not response or 'companies' not in response or not response['companies']:
                break
                
            all_companies.extend(response['companies'])
            
            if len(response['companies']) < per_page:
                break
                
            page += 1
            time.sleep(1)
        
        return all_companies
    
    def get_funnels(self, page=1, per_page=100):
        """Get all funnels"""
        params = {
            'page': page,
            'per_page': per_page
        }
        return self._make_request('/funnels', params=params)
    
    def get_all_funnels(self):
        """Get all funnels by paginating through results"""
        all_funnels = []
        page = 1
        per_page = 100
        
        while True:
            response = self.get_funnels(page=page, per_page=per_page)
            
            if not response or 'funnels' not in response or not response['funnels']:
                break
                
            all_funnels.extend(response['funnels'])
            
            if len(response['funnels']) < per_page:
                break
                
            page += 1
            time.sleep(1)
        
        return all_funnels
    
    def get_broadcasts(self, page=1, per_page=100):
        """Get all broadcasts"""
        params = {
            'page': page,
            'per_page': per_page
        }
        return self._make_request('/broadcasts', params=params)
    
    def get_all_broadcasts(self):
        """Get all broadcasts by paginating through results"""
        all_broadcasts = []
        page = 1
        per_page = 100
        
        while True:
            response = self.get_broadcasts(page=page, per_page=per_page)
            
            if not response or 'broadcasts' not in response or not response['broadcasts']:
                break
                
            all_broadcasts.extend(response['broadcasts'])
            
            if len(response['broadcasts']) < per_page:
                break
                
            page += 1
            time.sleep(1)
        
        return all_broadcasts
    
    def get_emails(self, page=1, per_page=100):
        """Get all emails"""
        params = {
            'page': page,
            'per_page': per_page
        }
        return self._make_request('/emails', params=params)
    
    def get_all_emails(self):
        """Get all emails by paginating through results"""
        all_emails = []
        page = 1
        per_page = 100
        
        while True:
            response = self.get_emails(page=page, per_page=per_page)
            
            if not response or 'emails' not in response or not response['emails']:
                break
                
            all_emails.extend(response['emails'])
            
            if len(response['emails']) < per_page:
                break
                
            page += 1
            time.sleep(1)
        
        return all_emails
    
    def get_sms(self, page=1, per_page=100):
        """Get all SMS messages"""
        params = {
            'page': page,
            'per_page': per_page
        }
        return self._make_request('/sms', params=params)
    
    def get_all_sms(self):
        """Get all SMS messages by paginating through results"""
        all_sms = []
        page = 1
        per_page = 100
        
        while True:
            response = self.get_sms(page=page, per_page=per_page)
            
            if not response or 'sms' not in response or not response['sms']:
                break
                
            all_sms.extend(response['sms'])
            
            if len(response['sms']) < per_page:
                break
                
            page += 1
            time.sleep(1)
        
        return all_sms
    
    def get_campaigns(self, page=1, per_page=100):
        """Get all campaigns"""
        params = {
            'page': page,
            'per_page': per_page
        }
        return self._make_request('/campaigns', params=params)
    
    def get_all_campaigns(self):
        """Get all campaigns by paginating through results"""
        all_campaigns = []
        page = 1
        per_page = 100
        
        while True:
            response = self.get_campaigns(page=page, per_page=per_page)
            
            if not response or 'campaigns' not in response or not response['campaigns']:
                break
                
            all_campaigns.extend(response['campaigns'])
            
            if len(response['campaigns']) < per_page:
                break
                
            page += 1
            time.sleep(1)
        
        return all_campaigns
    
    def get_superlinks(self, page=1, per_page=100):
        """Get all superlinks"""
        params = {
            'page': page,
            'per_page': per_page
        }
        return self._make_request('/superlinks', params=params)
    
    def get_all_superlinks(self):
        """Get all superlinks by paginating through results"""
        all_superlinks = []
        page = 1
        per_page = 100
        
        while True:
            response = self.get_superlinks(page=page, per_page=per_page)
            
            if not response or 'superlinks' not in response or not response['superlinks']:
                break
                
            all_superlinks.extend(response['superlinks'])
            
            if len(response['superlinks']) < per_page:
                break
                
            page += 1
            time.sleep(1)
        
        return all_superlinks
    
    def get_events(self, page=1, per_page=100):
        """Get contact events"""
        params = {
            'page': page,
            'per_page': per_page
        }
        return self._make_request('/events', params=params)
    
    def get_all_events(self):
        """Get all events by paginating through results"""
        all_events = []
        page = 1
        per_page = 100
        
        while True:
            response = self.get_events(page=page, per_page=per_page)
            
            if not response or 'events' not in response or not response['events']:
                break
                
            all_events.extend(response['events'])
            
            if len(response['events']) < per_page:
                break
                
            page += 1
            time.sleep(1)
        
        return all_events
    
    def export_data_to_csv(self, data_type, file_name=None):
        """
        Export data to CSV file
        
        Args:
            data_type (str): Type of data to export (contacts, tags, events, etc.)
            file_name (str, optional): Custom file name
            
        Returns:
            str: Path to the saved CSV file
        """
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        if not file_name:
            file_name = f"groundhogg_{data_type}_{timestamp}.csv"
        
        data = []
        
        # Map data_type to corresponding method
        data_type_method_map = {
            'contacts': self.get_all_contacts,
            'tags': self.get_all_tags,
            'companies': self.get_all_companies,
            'funnels': self.get_all_funnels,
            'broadcasts': self.get_all_broadcasts,
            'emails': self.get_all_emails,
            'sms': self.get_all_sms,
            'campaigns': self.get_all_campaigns,
            'superlinks': self.get_all_superlinks,
            'events': self.get_all_events,
        }
        
        if data_type not in data_type_method_map:
            raise ValueError(f"Unsupported data type: {data_type}")
        
        data = data_type_method_map[data_type]()
        
        if not data:
            print(f"No {data_type} data found to export")
            return None
        
        # Convert to DataFrame and save to CSV
        df = pd.DataFrame(data)
        df.to_csv(file_name, index=False)
        
        print(f"Exported {len(data)} {data_type} to {file_name}")
        return file_name


def main():
    """Main function to run the script"""
    # Replace these with your actual credentials
    site_url = "https://scholistico.com"
    token = "26a0293b0a4da82a960e6eb9effb4e31"
    public_key = "0bc1ccfccb63183f99cbf2620847b64f"
    
    # Check if credentials are provided
    if token == "your_groundhogg_token" or public_key == "your_groundhogg_public_key":
        print("Please update the script with your actual API credentials.")
        return
    
    # Initialize the API client
    api = GroundhoggAPI(site_url, token, public_key)
    
    # Create directory for exports
    export_dir = "groundhogg_exports"
    os.makedirs(export_dir, exist_ok=True)
    os.chdir(export_dir)
    
    # Test the API connection
    print("Testing API connection...")
    test_response = api.get_contacts(page=1, per_page=1)
    if not test_response:
        print("Could not connect to the Groundhogg API. Please check your credentials and try again.")
        return
    
    print("API connection successful!")
    
    # List of all data types to export
    data_types = [
        'contacts',
        'tags',
        'companies',
        'funnels',
        'broadcasts',
        'emails',
        'sms',
        'campaigns',
        'superlinks'
    ]
    
    # Export all data types
    print("Starting data export from Groundhogg...")
    
    for data_type in data_types:
        print(f"\nExporting {data_type}...")
        try:
            api.export_data_to_csv(data_type)
        except Exception as e:
            print(f"Error exporting {data_type}: {e}")
    
    print(f"\nAll data exported to directory: {export_dir}")


if __name__ == "__main__":
    main()

Testing API connection...
Making request to: https://scholistico.com/wp-json/gh/v3/contacts
API connection successful!
Starting data export from Groundhogg...

Exporting contacts...
Making request to: https://scholistico.com/wp-json/gh/v3/contacts
Making request to: https://scholistico.com/wp-json/gh/v3/contacts
Making request to: https://scholistico.com/wp-json/gh/v3/contacts
Making request to: https://scholistico.com/wp-json/gh/v3/contacts
Making request to: https://scholistico.com/wp-json/gh/v3/contacts
Making request to: https://scholistico.com/wp-json/gh/v3/contacts
Making request to: https://scholistico.com/wp-json/gh/v3/contacts
Making request to: https://scholistico.com/wp-json/gh/v3/contacts
Making request to: https://scholistico.com/wp-json/gh/v3/contacts
Making request to: https://scholistico.com/wp-json/gh/v3/contacts
Making request to: https://scholistico.com/wp-json/gh/v3/contacts
API request error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection 

In [ ]:
expot all 
1) contacts - GET /wp-json/gh/v3/tags
2) Tags
3) companies
4) funnels
5) broadcasts
6) Emails
7) SMS
8) campaigns
9) superlinks


In [5]:
import requests
import json
import time
from datetime import datetime
import pandas as pd
import os
import sys

class GroundhoggAPI:
    def __init__(self, site_url, token, public_key):
        """
        Initialize the Groundhogg API client
        
        Args:
            site_url (str): Your WordPress site URL (e.g., https://example.com)
            token (str): Your Groundhogg API token
            public_key (str): Your Groundhogg public key
        """
        self.base_url = f"{site_url.rstrip('/')}/wp-json/gh/v3"
        self.token = token
        self.public_key = public_key
        self.headers = {
            "Gh-Token": self.token,
            "Gh-Public-Key": self.public_key
        }
        
    def _make_request(self, endpoint, method="GET", params=None, data=None):
        """Make an API request to Groundhogg"""
        url = f"{self.base_url}/{endpoint.lstrip('/')}"
        
        try:
            print(f"Making request to: {url}")
            
            if method.upper() == "GET":
                response = requests.get(url, headers=self.headers, params=params)
            elif method.upper() == "POST":
                response = requests.post(url, headers=self.headers, json=data, params=params)
            elif method.upper() == "PUT":
                response = requests.put(url, headers=self.headers, json=data, params=params)
            elif method.upper() == "DELETE":
                response = requests.delete(url, headers=self.headers, params=params)
            else:
                raise ValueError(f"Unsupported HTTP method: {method}")
            
            if response.status_code == 404:
                print(f"Resource not found at: {url}")
                return None
                
            response.raise_for_status()  # Raise exception for HTTP errors
            return response.json()
        
        except requests.exceptions.RequestException as e:
            print(f"API request error: {e}")
            return None
    
    def get_count(self, data_type):
        """
        Get total count of items for a specific data type
        
        Args:
            data_type (str): Type of data (contacts, tags, broadcasts, emails, sms)
            
        Returns:
            int: Total count or 0 if unavailable
        """
        endpoint = f'/{data_type}/count'
        response = self._make_request(endpoint)
        
        if response and 'count' in response:
            return response['count']
        return 0
    
    def get_items(self, data_type, page=1, per_page=100):
        """
        Get items of a specific type with pagination
        
        Args:
            data_type (str): Type of data (contacts, tags, broadcasts, emails, sms)
            page (int): Page number for pagination
            per_page (int): Number of items per page
            
        Returns:
            dict: JSON response containing items
        """
        params = {
            'page': page,
            'per_page': per_page
        }
        return self._make_request(f'/{data_type}', params=params)
    
    def get_all_items(self, data_type, report_progress=True):
        """
        Get all items of a specific type by paginating through results
        
        Args:
            data_type (str): Type of data (contacts, tags, broadcasts, emails, sms)
            report_progress (bool): Whether to print progress to console
            
        Returns:
            list: List of all items
            int: Total items fetched
            int: Total items in the system
        """
        all_items = []
        page = 1
        per_page = 100
        key_name = data_type  # Default key name in response
        
        # Different endpoints return different JSON structures
        if data_type == 'broadcasts':
            key_name = 'broadcasts'
        elif data_type == 'emails':
            key_name = 'emails'
        elif data_type == 'sms':
            key_name = 'messages'
        
        # Get total count
        total_count = self.get_count(data_type)
        if report_progress:
            print(f"Found {total_count} total {data_type} to fetch")
        
        while True:
            if report_progress:
                print(f"Fetching {data_type} - page {page}...")
                
            response = self.get_items(data_type, page=page, per_page=per_page)
            
            if not response or key_name not in response or not response[key_name]:
                break
                
            items_in_page = response[key_name]
            all_items.extend(items_in_page)
            
            if report_progress:
                print(f"Fetched {len(all_items)} of {total_count} {data_type}")
            
            if len(items_in_page) < per_page:
                break
                
            page += 1
            time.sleep(0.5)  # Avoid hitting rate limits
        
        return all_items, len(all_items), total_count
    
    def export_data_to_csv(self, data_type, file_name=None):
        """
        Export data to CSV file
        
        Args:
            data_type (str): Type of data to export (contacts, tags, broadcasts, emails, sms)
            file_name (str, optional): Custom file name
            
        Returns:
            str: Path to the saved CSV file
            bool: Whether all items were fetched
        """
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        if not file_name:
            file_name = f"groundhogg_{data_type}_{timestamp}.csv"
        
        print(f"\n=== Exporting {data_type} ===")
        
        data, fetched_count, total_count = self.get_all_items(data_type)
        
        if not data:
            print(f"No {data_type} data found to export")
            return None, False
        
        # Convert to DataFrame and handle nested JSON
        df = pd.json_normalize(data)
        df.to_csv(file_name, index=False)
        
        all_fetched = fetched_count >= total_count
        
        if all_fetched:
            print(f"Success: All {fetched_count} {data_type} fetched and exported to {file_name}")
        else:
            print(f"Warning: {fetched_count} of {total_count} {data_type} fetched and exported to {file_name}. {total_count - fetched_count} pending to fetch.")
        
        return file_name, all_fetched


def main():
    """Main function to run the script"""
    # Replace these with your actual credentials
    site_url = "https://scholistico.com"
    token = "26a0293b0a4da82a960e6eb9effb4e31"
    public_key = "0bc1ccfccb63183f99cbf2620847b64f"
    
    # Check if credentials are provided
    if token == "your_groundhogg_token" or public_key == "your_groundhogg_public_key":
        print("Please update the script with your actual API credentials.")
        return
    
    # Initialize the API client
    api = GroundhoggAPI(site_url, token, public_key)
    
    # Create directory for exports
    export_dir = "groundhogg_exports1"
    os.makedirs(export_dir, exist_ok=True)
    os.chdir(export_dir)
    
    # Test the API connection
    print("Testing API connection...")
    test_response = api.get_items('contacts', page=1, per_page=1)
    if not test_response:
        print("Could not connect to the Groundhogg API. Please check your credentials and try again.")
        return
    
    print("API connection successful!")
    
    # Export all data types
    print("\nStarting data export from Groundhogg...\n")
    
    results = {}
    
    # Define data types to export
    data_types = ['broadcasts', 'emails', 'sms'] # 'contacts', 'tags', 
    
    for data_type in data_types:
        file_path, all_fetched = api.export_data_to_csv(data_type)
        results[data_type] = {
            'file': file_path,
            'all_fetched': all_fetched
        }
    
    # Print summary
    print("\n=== Export Summary ===")
    for data_type, result in results.items():
        status = "All fetched ✓" if result['all_fetched'] else "Incomplete ✗"
        file_info = result['file'] if result['file'] else "Not exported"
        print(f"{data_type.capitalize()}: {status} - {file_info}")
    
    print(f"\nAll data exported to directory: {export_dir}")


if __name__ == "__main__":
    main()

Testing API connection...
Making request to: https://scholistico.com/wp-json/gh/v3/contacts
API connection successful!

Starting data export from Groundhogg...


=== Exporting broadcasts ===
Making request to: https://scholistico.com/wp-json/gh/v3/broadcasts/count
Resource not found at: https://scholistico.com/wp-json/gh/v3/broadcasts/count
Found 0 total broadcasts to fetch
Fetching broadcasts - page 1...
Making request to: https://scholistico.com/wp-json/gh/v3/broadcasts
Fetched 248 of 0 broadcasts
Fetching broadcasts - page 2...
Making request to: https://scholistico.com/wp-json/gh/v3/broadcasts
Fetched 496 of 0 broadcasts
Fetching broadcasts - page 3...
Making request to: https://scholistico.com/wp-json/gh/v3/broadcasts
Fetched 744 of 0 broadcasts
Fetching broadcasts - page 4...
Making request to: https://scholistico.com/wp-json/gh/v3/broadcasts
Fetched 992 of 0 broadcasts
Fetching broadcasts - page 5...
Making request to: https://scholistico.com/wp-json/gh/v3/broadcasts
Fetched 124

KeyboardInterrupt: 

In [ ]:
?token=26a0293b0a4da82a960e6eb9effb4e31&public_key=0bc1ccfccb63183f99cbf2620847b64f    


token = "26a0293b0a4da82a960e6eb9effb4e31"
    public_key = "0bc1ccfccb63183f99cbf2620847b64f"

In [7]:
import requests
import pandas as pd
import time
import os

# API Authentication details
SITE_URL = "http://scholistico.com"
TOKEN = "26a0293b0a4da82a960e6eb9effb4e31"  # Replace with your actual token
PUBLIC_KEY = "0bc1ccfccb63183f99cbf2620847b64f"  # Replace with your actual public key

# Headers for authentication
HEADERS = {
    "GH-TOKEN": TOKEN,
    "GH-PUBLIC-KEY": PUBLIC_KEY
}

# Create output directory
os.makedirs("groundhogg_exports", exist_ok=True)

def fetch_all_pages(endpoint, item_key, total_count=None):
    """
    Fetch all pages of data from a paginated API endpoint
    
    Args:
        endpoint (str): API endpoint path
        item_key (str): Key in the response that contains the items
        total_count (int, optional): Expected total count for progress tracking
        
    Returns:
        list: All items from all pages
    """
    url = f"{SITE_URL}/wp-json/gh/v3/{endpoint}"
    all_items = []
    page = 1
    per_page = 100
    
    print(f"Fetching {endpoint}...")
    
    while True:
        params = {"page": page, "per_page": per_page}
        
        try:
            print(f"Fetching page {page}...")
            response = requests.get(url, headers=HEADERS, params=params)
            response.raise_for_status()
            data = response.json()
            
            # Check if we have items in the response
            if item_key not in data or not data[item_key]:
                break
            
            items = data[item_key]
            all_items.extend(items)
            
            # Show progress
            progress = f"Got {len(all_items)} out of {total_count}" if total_count else f"Got {len(all_items)} so far"
            print(f"  {progress}")
            
            # If we got fewer items than requested, we're on the last page
            if len(items) < per_page:
                break
                
            # Move to the next page
            page += 1
            
            # Add a small delay to be respectful to the API
            time.sleep(0.5)
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {endpoint}: {e}")
            break
    
    return all_items

def save_to_csv(items, filename):
    """Save items to a CSV file"""
    if not items:
        print(f"No items to save for {filename}")
        return
    
    file_path = f"groundhogg_exports/{filename}.csv"
    
    try:
        df = pd.DataFrame(items)
        df.to_csv(file_path, index=False)
        print(f"Saved {len(items)} items to {file_path}")
    except Exception as e:
        print(f"Error saving {filename}: {e}")

def main():
    print("Starting Groundhogg data export...")
    
    # Check if credentials are provided
    if TOKEN == "YOUR_GH_TOKEN_HERE" or PUBLIC_KEY == "YOUR_GH_PUBLIC_KEY_HERE":
        print("Please update the script with your actual TOKEN and PUBLIC_KEY values.")
        return
    
    # 1. Export Contacts (known total: 166068)
    contacts = fetch_all_pages("contacts", "contacts", 166068)
    save_to_csv(contacts, "contacts")
    
    # 2. Export Tags (known total: 181)
    tags = fetch_all_pages("tags", "tags", 181)
    save_to_csv(tags, "tags")
    
    # 3. Export Broadcasts (known total: 281)
    broadcasts = fetch_all_pages("broadcasts", "broadcasts", 281)
    save_to_csv(broadcasts, "broadcasts")
    
    # 4. Export Emails (known total: 489)
    emails = fetch_all_pages("emails", "emails", 489)
    save_to_csv(emails, "emails")
    
    # 5. Export SMS (known total: 23)
    sms = fetch_all_pages("sms", "sms", 23)
    save_to_csv(sms, "sms")
    
    print("Data export completed!")

if __name__ == "__main__":
    main()

Starting Groundhogg data export...
Fetching contacts...
Fetching page 1...
  Got 100 out of 166068
Fetching page 2...
  Got 200 out of 166068
Fetching page 3...
  Got 300 out of 166068
Fetching page 4...
  Got 400 out of 166068
Fetching page 5...
  Got 500 out of 166068
Fetching page 6...
  Got 600 out of 166068
Fetching page 7...
  Got 700 out of 166068
Fetching page 8...
  Got 800 out of 166068
Fetching page 9...
  Got 900 out of 166068
Fetching page 10...
  Got 1000 out of 166068
Fetching page 11...
  Got 1100 out of 166068
Fetching page 12...
  Got 1200 out of 166068
Fetching page 13...
  Got 1300 out of 166068
Fetching page 14...
  Got 1400 out of 166068
Fetching page 15...
  Got 1500 out of 166068
Fetching page 16...
  Got 1600 out of 166068
Fetching page 17...
  Got 1700 out of 166068
Fetching page 18...
  Got 1800 out of 166068
Fetching page 19...
  Got 1900 out of 166068
Fetching page 20...
  Got 2000 out of 166068
Fetching page 21...
  Got 2100 out of 166068
Fetching page 22.